In [16]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [27]:
from PIL import Image

# Replace 'path/to/your/image.jpg' with the actual path to your image
img_path = 'CM_Gender_All.png'

# Open the image using Pillow
img = Image.open(img_path)

# Save the image with a new filename and format (e.g., PNG)
output_path = 'CM_Gender_All.png'  # Replace with desired output path and filename
img.save(output_path)


In [17]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [18]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Trousers & Jeans - All - Train.npz')
data_validation = np.load(r'Full Dataset/Trousers & Jeans - All - Validation.npz')
data_test = np.load(r'Full Dataset/Trousers & Jeans - All - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['genders']

images_val = data_validation['images']
labels_val = data_validation['genders']


images_test = data_test['images']
labels_test = data_test['genders']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [19]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))


METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/ModelG 2/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [20]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(64,3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(64, activation = 'relu'),
                
                Dense(2)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/ModelG 2/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Male", "Female"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/ModelG 2/fit/run-{session_num}")


    return accuracy



In [21]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/ModelG 2/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
16/16 [==============================] - 1s 65ms/step
63/63 - 29s - loss: 0.7393 - accuracy: 0.6598 - val_loss: 0.4941 - val_accuracy: 0.7840 - 29s/epoch - 468ms/step
Epoch 2/15
16/16 [==============================] - 1s 61ms/step
63/63 - 27s - loss: 0.4095 - accuracy: 0.8207 - val_loss: 0.4155 - val_accuracy: 0.8140 - 27s/epoch - 421ms/step
Epoch 3/15
16/16 [==============================] - 1s 61ms/step
63/63 - 26s - loss: 0.3477 - accuracy: 0.8564 - val_loss: 0.4025 - val_accuracy: 0.8460 - 26s/epoch - 416ms/step
Epoch 4/15
16/16 [==============================] - 1s 62ms/step
63/63 - 27s - loss: 0.3298 - accuracy: 0.8564 - val_loss: 0.4130 - val_accuracy: 0.8140 - 27s/epoch - 426ms/step
Epoch 5/15
16/16 [==============================] - 1s 64ms/step
63/63 - 28s - loss: 0.3210 - accuracy: 0.8669 - val_loss: 0.4594 - val_accuracy: 0.8000 - 28s/epoch - 438ms/step
16/16 [==============================] - 1s 63ms

INFO:tensorflow:Assets written to: saved_model/ModelG 2/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
16/16 [==============================] - 1s 69ms/step
63/63 - 31s - loss: 0.6937 - accuracy: 0.7255 - val_loss: 0.4474 - val_accuracy: 0.8120 - 31s/epoch - 488ms/step
Epoch 2/15
16/16 [==============================] - 1s 62ms/step
63/63 - 28s - loss: 0.3736 - accuracy: 0.8447 - val_loss: 0.3952 - val_accuracy: 0.8460 - 28s/epoch - 439ms/step
Epoch 3/15
16/16 [==============================] - 1s 63ms/step
63/63 - 27s - loss: 0.3467 - accuracy: 0.8546 - val_loss: 0.3834 - val_accuracy: 0.8440 - 27s/epoch - 430ms/step
Epoch 4/15
16/16 [==============================] - 1s 63ms/step
63/63 - 27s - loss: 0.3184 - accuracy: 0.8589 - val_loss: 0.3950 - val_accuracy: 0.8320 - 27s/epoch - 430ms/step
Epoch 5/15
16/16 [==============================] - 1s 63ms/step
63/63 - 27s - loss: 0.2864 - accuracy: 0.8801 - val_loss: 0.3769 - val_accuracy: 0.8480 - 27s/epoch - 433ms/step
Epoch 6/15
16/16 [==============================

INFO:tensorflow:Assets written to: saved_model/ModelG 2/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 43ms/step
63/63 - 18s - loss: 0.6348 - accuracy: 0.7068 - val_loss: 0.4950 - val_accuracy: 0.7400 - 18s/epoch - 286ms/step
Epoch 2/15
16/16 [==============================] - 1s 37ms/step
63/63 - 18s - loss: 0.3954 - accuracy: 0.8272 - val_loss: 0.4174 - val_accuracy: 0.8380 - 18s/epoch - 288ms/step
Epoch 3/15
16/16 [==============================] - 1s 41ms/step
63/63 - 17s - loss: 0.3417 - accuracy: 0.8571 - val_loss: 0.3848 - val_accuracy: 0.8460 - 17s/epoch - 271ms/step
Epoch 4/15
16/16 [==============================] - 1s 39ms/step
63/63 - 19s - loss: 0.2993 - accuracy: 0.8799 - val_loss: 0.4526 - val_accuracy: 0.7840 - 19s/epoch - 299ms/step
Epoch 5/15
16/16 [==============================] - 1s 35ms/step
63/63 - 17s - loss: 0.2714 - accuracy: 0.8869 - val_loss: 0.4178 - val_accuracy: 0.8160 - 17s/epoch - 266ms/step
16/16 [==============================] - 1s 34ms

...

INFO:tensorflow:Assets written to: saved_model/ModelG 2/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
16/16 [==============================] - 1s 41ms/step
63/63 - 24s - loss: 0.6977 - accuracy: 0.6643 - val_loss: 0.5019 - val_accuracy: 0.7300 - 24s/epoch - 386ms/step
Epoch 2/15
16/16 [==============================] - 1s 39ms/step
63/63 - 21s - loss: 0.4183 - accuracy: 0.8174 - val_loss: 0.4338 - val_accuracy: 0.8280 - 21s/epoch - 329ms/step
Epoch 3/15
16/16 [==============================] - 1s 45ms/step
63/63 - 20s - loss: 0.3590 - accuracy: 0.8514 - val_loss: 0.4458 - val_accuracy: 0.7720 - 20s/epoch - 321ms/step
Epoch 4/15
16/16 [==============================] - 1s 57ms/step
63/63 - 21s - loss: 0.3148 - accuracy: 0.8611 - val_loss: 0.3641 - val_accuracy: 0.8420 - 21s/epoch - 341ms/step
Epoch 5/15
16/16 [==============================] - 1s 36ms/step
63/63 - 21s - loss: 0.2833 - accuracy: 0.8839 - val_loss: 0.3471 - val_accuracy: 0.8440 - 21s/epoch - 333ms/step
Epoch 6/15
16/16 [=============================

INFO:tensorflow:Assets written to: saved_model/ModelG 2/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
16/16 [==============================] - 1s 40ms/step
63/63 - 23s - loss: 0.6883 - accuracy: 0.6201 - val_loss: 0.5317 - val_accuracy: 0.7280 - 23s/epoch - 367ms/step
Epoch 2/15
16/16 [==============================] - 1s 41ms/step
63/63 - 20s - loss: 0.4501 - accuracy: 0.7975 - val_loss: 0.4680 - val_accuracy: 0.7680 - 20s/epoch - 321ms/step
Epoch 3/15
16/16 [==============================] - 1s 38ms/step
63/63 - 19s - loss: 0.3861 - accuracy: 0.8322 - val_loss: 0.4514 - val_accuracy: 0.7740 - 19s/epoch - 306ms/step
Epoch 4/15
16/16 [==============================] - 1s 39ms/step
63/63 - 19s - loss: 0.3346 - accuracy: 0.8569 - val_loss: 0.3570 - val_accuracy: 0.8640 - 19s/epoch - 308ms/step
Epoch 5/15
16/16 [==============================] - 1s 42ms/step
63/63 - 19s - loss: 0.3244 - accuracy: 0.8689 - val_loss: 0.3444 - val_accuracy: 0.8540 - 19s/epoch - 301ms/step
Epoch 6/15
16/16 [============================

INFO:tensorflow:Assets written to: saved_model/ModelG 2/fit/run-12\assets


In [24]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 2/hparam_tuning'


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelG 2/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
